# tests tab-NTV

In [6]:
from json_ntv import NtvSingle, NtvList, NtvSet, Ntv, NtvConnector
from json_ntv import NtvType, Namespace



In [4]:
fields = [{'full_dates::datetime': ['1964-01-01', '1985-02-05', '2022-01-21']},
          {'full_coord::point':    [[1,2], [3,4], [5,6]]},
          {'full_simple': [1,2,3,4]},
          {'complete_test': [['a', 'b'], [0, 0, 1, 0]]},
          {'complete_test': [['a', 'b'], [0, 0, 1, 0]]},
          {'implicit_test': [['a', 'b'], 'parent']},
          {'relative_test': [{'::string': ['a', 'b']}, 1, [0, 1, 1]]},
          [{'::string': ['a', 'b']}, 1, [0, 1, 1]],
          {'primary_test': [['a', 'b'], [2]]},
          [['a', 'b'], [2]],
          {'unic_test': 'valunic' },
          'valunic',
          {'primary': [['oui', 'fin 2022'], [1]]},
          [['oui', 'fin 2022'], [1]]]

field_old = [{'test': 'datvalue'}, ['a', 'b'], [1, [0, 0, 1, 0]]]

In [7]:
from observation import Iindex
def decode_ntv(field, encode_format='json'):
    '''return tuple : name, type, codec, parent, keys'''
    ntv = Ntv.obj(field)
    type_ntv = None
    if ntv.ntv_type:
        type_ntv = ntv.ntv_type.long_name
    if isinstance(ntv, NtvSingle):
        return (ntv.ntv_name, type_ntv, ntv.to_obj(simpleval=True, encode_format=encode_format), 
                None, None)
    if len(ntv) == 0 or len(ntv) > 3:
        return (ntv.ntv_name, type_ntv, ntv.to_obj(simpleval=True, encode_format=encode_format), 
                None, None)
    codec_ntv = ntv[0]
    type_codec_ntv = None
    if codec_ntv.ntv_type:
        type_codec_ntv = codec_ntv.ntv_type.long_name
    if len(ntv) == 3 and isinstance(ntv[1], NtvSingle) and isinstance(ntv[1].ntv_value, (int, str)) and \
      not isinstance(ntv[2], NtvSingle) and isinstance(ntv[2][0].ntv_value, int):
        return (ntv.ntv_name, type_codec_ntv, codec_ntv.to_obj(simpleval=True, encode_format=encode_format), 
                ntv[1].ntv_value, ntv[2].to_obj())
    if len(ntv) == 2 and isinstance(ntv[1], NtvSingle) and isinstance(ntv[1].ntv_value, (int, str)):
        return (ntv.ntv_name, type_codec_ntv, codec_ntv.to_obj(simpleval=True, encode_format=encode_format), 
                ntv[1].ntv_value, None) 
    if len(ntv) == 2 and not isinstance(ntv[1], NtvSingle) and isinstance(ntv[1][0].ntv_value, int):
        return (ntv.ntv_name, type_codec_ntv, codec_ntv.to_obj(simpleval=True, encode_format=encode_format), 
                None, ntv[1].to_obj())
    return (ntv.ntv_name, type_ntv, ntv.to_obj(simpleval=True, encode_format=encode_format), 
                None, None)

def from_ntv_iindex(ntv_value):
    name, typ, codec, parent, keys = decode_ntv(ntv_value, encode_format='json')
    if parent or (keys and len(keys) == 1):
        return None
    return Iindex(codec=codec, name=name, keys=keys, typevalue=None, reindex=True)
    
for field in fields:
    #print(decode_ntv(field, 'json'))
    print(from_ntv_iindex(field))

#print(ntv.ntv_name, codec_ntv.ntv_type, codec_ntv.to_obj(simpleval=True), parent_ntv, keys_ntv)    


    ["full_dates", ["1964-01-01T00:00:00+00:00", "1985-02-05T00:00:00+00:00", "2022-01-21T00:00:00+00:00"]]

    ["full_coord", [[1, 2], [3, 4], [5, 6]]]

    ["full_simple", [1, 2, 3, 4]]

    ["complete_test", ["a", "a", "b", "a"]]

    ["complete_test", ["a", "a", "b", "a"]]

None
None
None
None
None
    ["unic_test", ["valunic"]]

    ["valunic"]

None
None


In [18]:
tab_data   = {'dates::datetime': ['1964-01-01', '1985-02-05', '2022-01-21', '2022-01-22'], 
              'value':           [10, 20, 30, 40],
              'value32::int32':  [10, 20, 30, 40],
              'coord::point':    [[1,2], [3,4], [5,6], [7,8]],
              'names::string':   ['john', 'eric', 'judith', 'max'],
              'index':           [1, 2, 3, 4],
              'complete_test':   [['a', 'b'], [0, 0, 1, 0]],
              'implicit_test':   [['a', 'b'], [1]],
              'implicit_test2':  [['a', 'b'], [2]]}

from observation import Ilist
def from_ntv_ilist(ntv_value, reindex=False):
    ntv = Ntv.obj(ntv_value)
    leng = max([len(ntvi) for ntvi in ntv.ntv_value])

    # decode: name, type, codec, parent, keys
    lidx = [list(decode_ntv(ntvf)) for ntvf in ntv]
    for ind in range(len(lidx)):
        if lidx[ind][0] is None:
            lidx[ind][0] = 'i'+str(ind)
        _init_keys(ind, lidx, leng)
    lindex = [Iindex(idx[2], idx[0], idx[4], None, # idx[1],
                 reindex=reindex) for idx in lidx]
    return Ilist(lindex, reindex=False)


@staticmethod
def _init_keys(ind, lidx, leng):
    ''' initialization of keys data'''
    # name: 0, type: 1, codec: 2, parent: 3, keys: 4
    if lidx[ind][4] is None and lidx[ind][3] is None:  #full
        lidx[ind][4] = list(range(leng))
        return
    if lidx[ind][4] and len(lidx[ind][4]) > 1 and lidx[ind][3] is None:  #complete
        return
    if len(lidx[ind][2]) == 1: #unique
        lidx[ind][4] = [0] * leng
        return
    keys = lidx[ind][4]
    coef = keys[0]
    if lidx[ind][4] and len(lidx[ind][4]) == 1:  #primary
        lidx[ind][4] = [ (ikey % (coef * len(lidx[ind][2]))) // coef for ikey in range(leng)]
        print(coef, len(lidx[ind][2]), leng, lidx[ind][4])
        return  
    par_coe = lidx[ind][3]
    if par_coe is None:
        raise IlistError('keys not referenced')
          
    if not lidx[par_coe][4] or len(lidx[par_coe][4]) != leng:
        _init_keys(par_coe, lidx, leng)
    if not lidx[ind][4] and len(lidx[ind][2]) == len(lidx[par_coe][2]):    # implicit
        lidx[ind][4] = lidx[par_coe][4]
        return
    # derived keys
    lidx[ind][4] = Iindex.keysfromderkeys(lidx[par_coe][4], lidx[ind][4])
    return
    
print(from_ntv_ilist(tab_data))

1 2 4 [0, 1, 0, 1]
2 2 4 [0, 0, 1, 1]
    ["dates", ["1964-01-01T00:00:00+00:00", "1985-02-05T00:00:00+00:00", "2022-01-21T00:00:00+00:00", "2022-01-22T00:00:00+00:00"]]
    ["complete_test", ["a", "a", "b", "a"]]
    ["value", [10, 20, 30, 40]]
    ["value32", [10, 20, 30, 40]]
    ["coord", [[1, 2], [3, 4], [5, 6], [7, 8]]]
    ["names", ["john", "eric", "judith", "max"]]
    ["index", [1, 2, 3, 4]]

    ["implicit_test", ["a", "b", "a", "b"]]
    ["implicit_test2", ["a", "a", "b", "b"]]



In [ ]:

tab_data   = {'dates::datetime': ['1964-01-01', '1985-02-05', '2022-01-21'], 
              'value':           [10, 20, 30],
              'value32::int32':  [10, 20, 30],
              'coord::point':    [[1,2], [3,4], [5,6]],
              'names::string':   ['john', 'eric', 'judith'],
              'index':           [1, 2, 3]}

field = Ntv.obj({':field': field_data})
fiel2 = Ntv.obj({':field': field_dat2})
tab   = Ntv.obj({':tab'   : tab_data})

# the DataFrame Connector is associated with NtvType 'tab' in dicobj 
sr = fiel2.to_obj(encode_format='obj', dicobj={'field': 'SeriesConnec'})
df = tab.to_obj  (encode_format='obj', dicobj={'tab': 'DataFrameConnec'})

# pandas dtype conform to Ntv type
print(sr, '\n')
print(df.dtypes)
df

In [ ]:
import json
type_to_dtype = {'date': 'datetime64[ns]', 'datetime': 'datetime64[ns]', 'string': 'string', 'int32': 'int32', 'int64': 'int64' }
dtype_to_type = {'datetime64[ns]': 'datetime', 'string': 'string', 'int32': 'int32', 'int64': 'json'}
#print(dtype_to_type)

def to_ntv_sr(sr):
    ntv_type = None
    dtype = sr.dtype.name
    if dtype in dtype_to_type:
        if dtype_to_type[dtype] != 'json':
            ntv_type = dtype_to_type[dtype]
        ntv_value = json.loads(sr.to_json(orient='records', date_format='iso', default_handler=str))
    elif sr.dtype.name == 'object':
        ntv_value = sr.to_list()
    ntv_name = sr.name
    return NtvSingle(NtvList(ntv_value, ntv_name, ntv_type).to_obj(), None, 'field')
    
def to_ntv_df(df):
    df2 = df.reset_index()
    return NtvSingle(NtvSet([to_ntv_sr(df2[colon]).ntv_value for colon in df2.columns]).to_obj(), None, 'tab')
    #print([df[colon].to_list() for colon in df.columns])

In [ ]:
print(to_ntv_sr(df.value).to_obj(), '\n')
print(to_ntv_df(df))

In [ ]:
sr = from_ntv_sr(tab[0])
sr2 = from_ntv_sr(tab[3])
df = from_ntv_df(tab)
print(sr.name, sr.dtype, type(sr.dtype), sr2.name, sr2.dtype)
print(df.dtypes)
print(to_ntv_sr(df['dates']))
print(to_ntv_df(df))

In [ ]:
lis = [df[]]
print(df)

In [ ]:
print(Series(['john', 'eric', 'judith'], dtype='string'))
print(tab[3])
print(tab[3].to_obj(encode_format='obj', simpleval=True))